# Word2Vec (Skipgram )

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
import nltk
from nltk.corpus import semcor, stopwords

nltk.download('semcor')
nltk.download('stopwords')

[nltk_data] Downloading package semcor to
[nltk_data]     C:\Users\surya\AppData\Roaming\nltk_data...
[nltk_data]   Package semcor is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\surya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
torch.cuda.is_available(), torch.cuda.get_device_name(0)

(True, 'NVIDIA GeForce RTX 2080 with Max-Q Design')

In [4]:
semcor.words()

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]

## 1. Load data

In [5]:
#1. tokenization
corpus = semcor.sents()

In [6]:
stop_words = set(stopwords.words('english'))
corpus = [[word for word in sent if word.lower() not in stop_words] for sent in corpus]

# Remove punctuation from corpus
from string import punctuation
corpus = [[word for word in sent if word not in punctuation] for sent in corpus]

# Remove empty sentences
corpus = [sent for sent in corpus if len(sent) > 0]

# Remove sentences with less than 5 words
corpus = [sent for sent in corpus if len(sent) >= 5]

# Remove sentences with more than 20 words
corpus = [sent for sent in corpus if len(sent) <= 20]


In [7]:
#2. numeralization
#find unique words
flatten = lambda l: [item for sublist in l for item in sublist]
#assign unique integer
vocabs = list(set(flatten(corpus))) #all the words we have in the system - <UNK>

In [8]:
#create handy mapping between integer and word
word2index = {v:idx for idx, v in enumerate(vocabs)}


In [9]:
vocabs.append('<UNK>')
word2index['<UNK>'] = len(vocabs) - 1  

In [10]:
index2word = {v:k for k, v in word2index.items()}

## 2. Prepare train data

In [11]:
#create pairs of center word, and outside word

def random_batch(batch_size, corpus, window_size=2):

    skipgrams = []

    #loop each corpus
    for doc in corpus:
        #look from the 2nd word until second last word
        for i in range(window_size, len(doc)-window_size):
            #center word
            center = word2index[doc[i]]
            
            #outside words (window size on both sides of the center word)
            outside_start =  i - window_size
            outside_end =  i + window_size + 1 # +1 because the end index is exclusive

            # outside = []
            # loop through the outside words, append to the list 'outside'
            for j in range(outside_start, outside_end):
                if i != j:  # Skip the center word
                    outside= word2index[doc[j]]
                    skipgrams.append([center, outside])

                
    random_index = np.random.choice(range(len(skipgrams)), batch_size, replace=False)
    
    inputs, labels = [], []
    for index in random_index:
        inputs.append([skipgrams[index][0]])
        labels.append([skipgrams[index][1]])
        
    return np.array(inputs), np.array(labels)
            
x, y = random_batch(2, corpus)

In [12]:
x.shape  #batch_size, 1

(2, 1)

In [13]:
x

array([[25398],
       [ 2257]])

In [14]:
y.shape  #batch_size 1

(2, 1)

## 3. Model

$$J(\theta) = -\frac{1}{T}\sum_{t=1}^{T}\sum_{\substack{-m \leq j \leq m \\ j \neq 0}}\log P(w_{t+j} | w_t; \theta)$$

where $P(w_{t+j} | w_t; \theta) = $

$$P(o|c)=\frac{\exp(\mathbf{u_o^{\top}v_c})}{\sum_{w=1}^V\exp(\mathbf{u_w^{\top}v_c})}$$

where $o$ is the outside words and $c$ is the center word

$$P(o|c)=\frac{\exp(\mathbf{u_o^{\top}v_c})}{\sum_{w=1}^V\exp(\mathbf{u_w^{\top}v_c})}$$

In [18]:
class Skipgram(nn.Module):
    
    def __init__(self, voc_size, emb_size):
        super(Skipgram, self).__init__()
        self.embedding_center  = nn.Embedding(voc_size, emb_size)
        self.embedding_outside = nn.Embedding(voc_size, emb_size)
    
    def forward(self, center, outside, all_vocabs):
        center_embedding     = self.embedding_center(center)  #(batch_size, 1, emb_size)
        outside_embedding    = self.embedding_center(outside) #(batch_size, 1, emb_size)
        all_vocabs_embedding = self.embedding_center(all_vocabs) #(batch_size, voc_size, emb_size)
        
        top_term = torch.exp(outside_embedding.bmm(center_embedding.transpose(1, 2)).squeeze(2))
        #batch_size, 1, emb_size) @ (batch_size, emb_size, 1) = (batch_size, 1, 1) = (batch_size, 1) 

        lower_term = all_vocabs_embedding.bmm(center_embedding.transpose(1, 2)).squeeze(2)
        #batch_size, voc_size, emb_size) @ (batch_size, emb_size, 1) = (batch_size, voc_size, 1) = (batch_size, voc_size) 
        
        lower_term_sum = torch.sum(torch.exp(lower_term), 1)  #(batch_size, 1)
        
        loss = -torch.mean(torch.log(top_term / lower_term_sum))  #scalar
        
        return loss
        

In [19]:
#prepare all vocabs

batch_size = 2
voc_size   = len(vocabs)

def prepare_sequence(seq, word2index):
    idxs = list(map(lambda w: word2index[w] if word2index.get(w) is not None else word2index["<UNK>"], seq))
    return torch.LongTensor(idxs)

all_vocabs = prepare_sequence(list(vocabs), word2index).expand(batch_size, voc_size)
all_vocabs

tensor([[    0,     1,     2,  ..., 36637, 36638, 36639],
        [    0,     1,     2,  ..., 36637, 36638, 36639]])

In [20]:
model = Skipgram(voc_size, 2)
model

Skipgram(
  (embedding_center): Embedding(36640, 2)
  (embedding_outside): Embedding(36640, 2)
)

In [21]:
input_tensor = torch.LongTensor(x)
label_tensor = torch.LongTensor(y)

In [22]:
loss = model(input_tensor, label_tensor, all_vocabs)

In [23]:
loss

tensor(10.7479, grad_fn=<NegBackward0>)

## 4. Training

In [25]:
batch_size = 2
emb_size   = 2
model      = Skipgram(voc_size, emb_size)
optimizer  = optim.Adam(model.parameters(), lr=0.001)

In [26]:
import time

def log_epoch(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [27]:
start = time.time()
num_epochs = 1000
window_size = 5

for epoch in range(num_epochs):
    
    #get batch
    input_batch, label_batch = random_batch(batch_size, corpus, window_size)
    input_tensor = torch.LongTensor(input_batch)
    label_tensor = torch.LongTensor(label_batch)
    
    #predict
    loss = model(input_tensor, label_tensor, all_vocabs)
    
    #backprogate
    optimizer.zero_grad()
    loss.backward()
    
    #update alpha
    optimizer.step()
    # Log epoch time
    epoch_mins, epoch_secs = log_epoch(start, time.time())
    
    #print the loss
    if (epoch + 1) % 100 == 0:
        print(f"Epoch: {epoch + 1} | Loss: {loss:.6f} | Time: {epoch_mins}m {epoch_secs}s")

Epoch: 100 | Loss: 12.330386 | Time: 1m 5s
Epoch: 200 | Loss: 10.850671 | Time: 2m 11s
Epoch: 300 | Loss: 11.087509 | Time: 3m 22s
Epoch: 400 | Loss: 14.636918 | Time: 4m 34s
Epoch: 500 | Loss: 11.455265 | Time: 5m 40s
Epoch: 600 | Loss: 9.962486 | Time: 6m 50s
Epoch: 700 | Loss: 12.481160 | Time: 7m 59s
Epoch: 800 | Loss: 10.493835 | Time: 9m 10s
Epoch: 900 | Loss: 12.907803 | Time: 10m 19s
Epoch: 1000 | Loss: 11.106666 | Time: 11m 29s


## 5. Plot the embeddings

Is fruit really near to banana?
Is fruit really far from cat?

In [28]:
vocabs

['recluse',
 'unlamented',
 'soldiery',
 'commonwealth',
 'Gettysburg',
 'stuck-up',
 'groove',
 'maria',
 'free',
 'mates',
 'Brittany',
 'Hunters',
 'stay',
 'blaze',
 'custodian',
 'replied',
 'entrenched',
 'allowances',
 'understood',
 'unreflective',
 'stagnation',
 'Ziminska-Sygietynska',
 'obscures',
 'misbehavior',
 'Rebel',
 'impasse',
 'driver',
 'speeds',
 'bunt',
 'exhibiting',
 'incipience',
 'assaulted',
 'statutes',
 'Spahn',
 'experimented',
 'Forum',
 '1863',
 'warfront',
 'controversy',
 'indirect',
 'snowfall',
 'Nowadays',
 'Durlach',
 'independence',
 'Quell',
 'Lehmann',
 'parisology',
 'Phoenix',
 'underside',
 'life',
 'legged',
 'archives',
 'dynasty',
 'autism',
 'shoelace',
 'Tradition',
 'materiel',
 'airmen',
 'distinction',
 'Racine',
 'refused',
 'totally',
 'vivo',
 'Rolls',
 'vivify',
 'Mister',
 "mind's",
 'multipurpose',
 'Micelles',
 'Junction',
 'Grand',
 'billets',
 'inane',
 'elucidated',
 'Bulletin',
 'disproving',
 'ugliness',
 'tranquilizers',

In [29]:
banana = torch.LongTensor([word2index['banana']])
banana

tensor([28528])

In [30]:
banana_embed_c = model.embedding_center(banana)
banana_embed_o = model.embedding_outside(banana)
banana_embed   = (banana_embed_c + banana_embed_o) / 2
banana_embed

tensor([[-0.2721,  0.3790]], grad_fn=<DivBackward0>)

In [31]:
banana_embed_o

tensor([[-0.6656,  1.0426]], grad_fn=<EmbeddingBackward0>)

In [32]:
def get_embed(word):
    try:
        index = word2index[word]
    except:
        index = word2index['<UNK>']
        
    word = torch.LongTensor([word2index[word]])
    
    embed_c = model.embedding_center(word)
    embed_o = model.embedding_outside(word)
    embed   = (embed_c + embed_o) / 2
    
    return embed[0][0].item(), embed[0][1].item()

In [33]:
get_embed('fruit')

(0.4043092429637909, -0.9147692322731018)

In [34]:
get_embed('cat')

(0.03199125453829765, 0.4378172755241394)

In [35]:
get_embed('dog')

(-0.28775718808174133, 0.9247953295707703)

In [36]:
get_embed('banana')

(-0.2720649540424347, 0.37896615266799927)

## 6. Cosine similarity

In [37]:
banana = get_embed('banana')
banana

(-0.2720649540424347, 0.37896615266799927)

In [38]:
fruit = get_embed('fruit')
fruit

(0.4043092429637909, -0.9147692322731018)

In [39]:
unk = get_embed('<UNK>')
unk

(-0.4432022273540497, -0.4926328659057617)

In [40]:
np.array(banana) @ np.array(unk)

-0.06611138825353269

In [42]:
#more formally is to divide by its norm
def cosine_similarity(A, B):
    dot_product = np.dot(A, B)
    norm_a = np.linalg.norm(A)
    norm_b = np.linalg.norm(B)
    similarity = dot_product / (norm_a * norm_b)
    return similarity

print(cosine_similarity(np.array(banana), np.array(unk)))
print(cosine_similarity(np.array(banana), np.array(fruit)))

-0.21385672431033478
-0.9787579100776338
